# Estimating Phosim CPU Times: 
## What can we learn from Twinkles Run 1?

Twinkles Run 1 is 1227 visits, a subset of the observations of a Deep Drilling Field from the kraken-1042 Opsim simulation.  Each of the visits is simulated as a 30-second observation with Phosim.  Together, the 1227 runs of Phosim required about 4 CPU years on the SLAC batch farm.  The CPU time required per visit varied widely.  Some required only a few hours.  More than 100 visits (representing about one third of the total CPU time) never completed because they reached the 5-day CPU time limit for the batch hosts, and produced no output.  

Studying the dependence of the CPU time on the various inputs to the Phosim simulations has several motivations.  Certainly there's no sense in investing CPU time in a run that cannot finish with 5 CPU days.  And Run 1 was relatively modest in terms of resource requirements.  Essentially all 1227 simulations ran in parallel on the batch farm.  For Twinkles1 and 2, which will be much larger, the batch farm will not be able to run all of the simulations in parallel and the effective throughput could well be determined by the individual simulations that take the most CPU time, if they are allowed to hog the available hosts. Twinkles1 and 2 might run at NERSC, which has more hosts, but reportedly a 2-day CPU limit per host.  Absent some solution involving checkpointing, we may need to make judicious adjustments to the Phosim inputs for these runs.  Also, John Peterson has pointed out that we should not assume that all of the inputs to Phosim for Run 1 were sensible.

For this study, a number of Phosim inputs were extracted from the instance catalogs for the runs, and combined with execution time information from the batch farm.  The columns of the resulting csv file are described [here](https://github.com/DarkEnergyScienceCollaboration/Twinkles/issues/159).

This Notebook is based on Phil Marshall's [machine learning example](https://github.com/drphilmarshall/StatisticalMethods/blob/master/examples/SDSScatalog/Quasars.ipynb).

In [ ]:
# For pretty plotting
# !pip install --upgrade seaborn

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
%pylab inline
import seaborn as sns
sns.set()
import copy
from __future__ import print_function

## Reading the data

* This is a regression problem, to be able to predict the CPU time in terms of various metadata related to the runs.  These apparently are called "features" in machine learning terminology.  Also the CPU time will be referred to as the "response variable."

* Read in the data.  Here just selected columns are being read from the file.  *They may or may not be the most relevant.  Also, the filter and hostname columns would need to be converted to numerical values in order to be used.  And the runlimit column probably needs special treatment because it is clearly related to large CPU times*

In [ ]:
run1meta = pd.read_csv("http://www.slac.stanford.edu/~digel/lsst/run1_metadata.csv",index_col=0,usecols=["obshistid","expmjd","filter","rotskypos","altitude",\
                                                "rawseeing","airmass","sunalt","moonalt","moonphase","dist2moon","cputime",\
                                                "runlimit"])

# Omit the two runs that have not actually finished running (or been 
# terminated at the CPU time limit).  These are flagged with -999.0 in
# the input file.  Also omit the runs that reached the CPU limit:
run1meta = run1meta[(run1meta["cputime"] > 0.) & (run1meta["runlimit"] == 0)]

#run1meta = run1meta[(run1meta["cputime"] > 0.)]


# Response variables: redshift
cputime = run1meta["cputime"]

# Features or attributes: photometric measurements
run1meta_features = copy.copy(run1meta)
del run1meta_features["cputime"]
del run1meta_features["runlimit"]
run1meta_features.head()

In [ ]:
bins =  hist(cputime.values,bins=100) ; xlabel("CPU time (s)") ; ylabel("N")

The distribution of CPU time has a very long tail, plus a number of runs piled up at the 5-day CPU time limit.

Let's plot all the features, colored by the CPU time, to look for structure.  (Possibly the logarithm of the CPU time should be used.)

In [ ]:
import matplotlib as mpl
import matplotlib.cm as cm

# Truncate the color at 3e5 CPU sec just to keep some contrast.
norm = mpl.colors.Normalize(vmin=min(cputime.values), vmax=300000)
cmap = cm.jet_r
m = cm.ScalarMappable(norm=norm, cmap=cmap)

# Plot everything against everything else:
rez = pd.scatter_matrix(run1meta_features,alpha=0.2,figsize=[15,15],color=m.to_rgba(cputime.values))

Now we have our machine learning inputs and outputs:

In [ ]:
X = run1meta_features.values  # Data: 5-d feature space
y = cputime.values # Target: redshifts

In [ ]:
print("Design matrix shape =", X.shape)
print("Response variable vector shape =", y.shape)

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

### Linear Regression

Let's follow the same procedure as in the [`SciKit-Learn` tutorial](../../scikit-learn/Linear_Regression.ipynb) we just went through:

In [ ]:
from sklearn import linear_model
linear = linear_model.LinearRegression()

# Fit the model, using all the attributes:
linear.fit(X_train, y_train)

# Do the prediction on the test data:
y_lr_pred = linear.predict(X_test)

# How well did we do?
from sklearn.metrics import mean_squared_error
mse_linear = np.sqrt(mean_squared_error(y_test,y_lr_pred))
r2_linear = linear.score(X_test, y_test)
print("Linear regression: MSE = ",mse_linear)
print("R2 score =",r2_linear)

In [ ]:
plot(y_test,y_lr_pred - y_test,'o',alpha=0.2)
title("Linear Regression Residuals - MSE = %.2f" % mse_linear)
xlabel("CPU Time")
ylabel("Residual")
ylim(-300000,200000)
hlines(0,min(y_test),max(y_test),color="red")

Just how bad is this? Here's the MSE from guessing the *average redshift of the training set* for all new objects:

In [ ]:
print("Naive MSE", ((1./len(y_train))*(y_train - y_train.mean())**2).sum())
print("Linear regression: MSE = ",mse_linear)

In [ ]:
mean_squared_error?

### *k*-Nearest Neighbor (KNN) Regression

Now let's try a different kind of model: a *non-parametric* one.

["Regression based on k-nearest neighbors. The target is predicted by local interpolation of the targets associated of the nearest neighbors in the training set."](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)


#### Question:

What underlying model is implied by the KNN algorithm? How many hidden parameters does it have?

In [ ]:
from sklearn import neighbors
from sklearn import preprocessing

X_scaled_knn = preprocessing.scale(X) # Many methods work better on scaled X.

KNN = neighbors.KNeighborsRegressor(5)

X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_scaled_knn, y)

KNN.fit(X_train_knn,y_train_knn)

In [ ]:
y_knn_pred = KNN.predict(X_test_knn)
mse_knn = mean_squared_error(y_test_knn,y_knn_pred)
r2_knn = KNN.score(X_test_knn, y_test_knn)
print("MSE (KNN) =", mse_knn)
print("R2 score (KNN) =",r2_knn)
print("cf.")
print("MSE (linear regression) = ",mse_linear)
print("R2 score (linear regression) =",r2_linear)

In [ ]:
plot(y_test_knn, y_knn_pred - y_test_knn,'o',alpha=0.2)
#plot(y_test, y_lr_pred - y_test,'x')
title("k-NN Residuals - MSE = %.2f" % mse_knn)
xlabel("CPU Time")
ylabel("Residual")
ylim(-300000,200000)
hlines(0,min(y_test),max(y_test),color="red")

### Tuning the KNN Model

* Let's vary the control parameters of the KNN model, to see how good we can make our predictions.

* We can see our options in the model `repr`:

> KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_neighbors=5, p=2, weights='uniform')

* Let's first make a "validation curve" to investigate one parameter: the number of nearest neighbors averaged over.

In [ ]:
# We'll vary the number of neighbors used:
param_name = "n_neighbors"
param_range = np.array([1,2,4,8,16,32,64])

# And we'll need a cv iterator:
from sklearn.cross_validation import ShuffleSplit
shuffle_split = ShuffleSplit(len(X), 10, test_size=0.4)

# Compute our cv scores for a range of the no. of neighbors:
from sklearn.learning_curve import validation_curve
training_scores, validation_scores = validation_curve(KNN, X_scaled, y,
                                                      param_name=param_name,
                                                      param_range=param_range, 
                                                      cv=shuffle_split, scoring='r2')

In [ ]:
def plot_validation_curve(param_name,parameter_values, training_scores, validation_scores):
    training_scores_mean = np.mean(training_scores, axis=1)
    training_scores_std = np.std(training_scores, axis=1)
    validation_scores_mean = np.mean(validation_scores, axis=1)
    validation_scores_std = np.std(validation_scores, axis=1)

    plt.fill_between(parameter_values, training_scores_mean - training_scores_std,
                     training_scores_mean + training_scores_std, alpha=0.1, color="r")
    plt.fill_between(parameter_values, validation_scores_mean - validation_scores_std,
                     validation_scores_mean + validation_scores_std, alpha=0.1, color="g")
    plt.plot(parameter_values, training_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(parameter_values, validation_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.ylim(validation_scores_mean.min() - .1, training_scores_mean.max() + .1)
    plt.xlabel(param_name)
    plt.legend(loc="best")

In [ ]:
plot_validation_curve(param_name, param_range, training_scores, validation_scores)

#### Question:

Can you explain the shapes of these two curves? Talk to your neighbor for a few minutes, and be prepared to suggest reasons for a) the rise and fall of the cross validation score and b) the monotonic decrease in training score.

#### Model tuning with `GridSearchCV`

* Now, let's see if we can do better by varying some other KNN options as well - in a *grid search*.

In [ ]:
param_grid = {'n_neighbors': np.array([1,2,4,8,16,32,64]),
                  'weights': ['uniform','distance'],
                       'p' : np.array([1,2])}

np.set_printoptions(suppress=True)
print(param_grid)

In [ ]:
from sklearn.grid_search import GridSearchCV
KNN_tuned = GridSearchCV(KNN, param_grid, verbose=3)

A `GridSearchCV` object behaves just like a model, except it carries out a cross-validation while fitting:

<img src="../../scikit-learn/figures/grid_search_cross_validation.svg" width=100%>

In [ ]:
KNN_tuned.fit(X_train, y_train)

In [ ]:
y_knn_tuned_pred = KNN_tuned.predict(X_test)

mse_knn_tuned = mean_squared_error(y_test,y_knn_tuned_pred)
r2_knn_tuned = KNN_tuned.score(X_test, y_test)

print("MSE (tuned KNN) =", mse_knn_tuned)
print("R2 score (tuned KNN) =",r2_knn_tuned)
print("cf.")
print("MSE (KNN) = ",mse_knn)
print("R2 score (KNN) =",r2_knn)

Which are the best KNN control parameters we found?

In [ ]:
KNN_tuned.best_params_

This value of `n_neighbors` is consistent with the peak in cross-validation score in the validation curve plot.

#### Generalization Error

Notice that all the above tuning happened while training on a single split (`X_train` and `y_train`).


It's possible that that particular fold prefers a slightly different set of parameters than a different one - so to assess our generalization error, we need a further level of cross-validation.


We can do this by passing a `GridSearchCV` model to the cross validation score calculator. This will take a few moments, as the grid search is carried out for each CV fold...

In [ ]:
from sklearn.cross_validation import cross_val_score

R2 = cross_val_score(KNN_tuned, X_scaled, y, cv=shuffle_split, scoring='r2')

In [ ]:
meanR2,errR2 = np.mean(R2),np.std(R2)
print("Mean score:",meanR2,"+/-",errR2)

### Notes

* Optimizing over control parameters (or hyper parameters) with grid search cross validation is a form of model selection.


* When presented with new metadata samples, and asked to predict the target response variables (CPU time), we'll need a trained machine that has not been *over-fitted* to the training data.


* Minimizing and estimating the generalization error is a way to reduce the risk of getting this prediction wrong. 


* Let's finish off our CPU time machine-learning algorithm.

In [ ]:
KNNz = KNN_tuned.best_estimator_
KNNz.fit(X_train, y_train)

In [ ]:
j = 71
one_pretend_run = X_test[j,:]
cpupredicted = KNNz.predict(one_pretend_run)
cpuactual = y_test[j]
print("True CPU cf. KNN predicted CPU time:",cpuactual,cpupredicted)

In [ ]:
cpuactual = y_test
cpupredicted = KNNz.predict(X_test)

plot(cpuactual, cpupredicted,'o',alpha=0.1)
title("KNNz performance")
xlabel("CPU Time")
ylabel("Predicted CPU Time")
lims = [0.0,450e3]
xlim(lims)
ylim(lims)
plot(lims, lims, ':k')

## Quasar Classification with Random Forests


* Let's switch gears and do a 3-class classification problem: star, galaxy, or QSO.


* A very good general-purpose classification (and regression!) algorithm is Random Forest. See [this blog post](http://blog.yhathq.com/posts/random-forests-in-python.html) for a nice high level introduction.


* ["A random forest is a meta estimator that fits a number of *decision tree classifiers* on various sub-samples of the dataset, and uses averaging to improve the predictive accuracy and control over-fitting.](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)


* Let's read in equal numbers of all three types of data, clean them up, and set $y$ equal to the classification label.

In [ ]:
all_sources = pd.read_csv("data/qso10000.csv",index_col=0,usecols=["objid","dered_r","u_g_color",\
                                                "g_r_color","r_i_color","i_z_color","diff_u",\
                                                "diff_g1","diff_i","diff_z","class"])[:1000]

all_sources = all_sources.append(pd.read_csv("data/star1000.csv",index_col=0,usecols=["objid","dered_r","u_g_color",\
                                                "g_r_color","r_i_color","i_z_color","diff_u",\
                                                "diff_g1","diff_i","diff_z","class"]))

all_sources = all_sources.append(pd.read_csv("data/galaxy1000.csv",index_col=0,usecols=["objid","dered_r","u_g_color",\
                                                "g_r_color","r_i_color","i_z_color","diff_u",\
                                                "diff_g1","diff_i","diff_z","class"]))

all_sources = all_sources[(all_sources["dered_r"] > -9999) & (all_sources["g_r_color"] > -10) & (all_sources["g_r_color"] < 10)]

all_labels = all_sources["class"]

all_features = copy.copy(all_sources)
del all_features["class"]

X = copy.copy(all_features.values)
y = copy.copy(all_labels.values)

In [ ]:
all_labels.tail()

In [ ]:
print("Feature vector shape =", X.shape)
print("Class label vector shape =", y.shape)

What structure can we see in the data? Let's plot all the features as before.

In [ ]:
yy = all_labels.values.copy()
yy[yy=="QSO"] = 0.0    # Red
yy[yy=="STAR"] = 0.5   # Green
yy[yy=="GALAXY"] = 1.0 # Blue

norm = mpl.colors.Normalize(vmin=min(yy), vmax=max(yy))
cmap = cm.jet_r
m = cm.ScalarMappable(norm=norm, cmap=cmap)
rez = pd.scatter_matrix(all_features,alpha=0.2,figsize=[15,15],color=m.to_rgba(yy))

OK - looks like there is information there to be used! 
Let's turn on the machine learning.

### Random Forest Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100,oob_score=True)
rf.fit(X,y)

What are the important features in the data?

In [ ]:
sorted(zip(all_sources.columns.values,rf.feature_importances_),key=lambda q: q[1],reverse=True)

In [ ]:
rf.oob_score_

This is the "Out of Bag" accuracy (of predicted y compared to truth), made available by ensemble classifiers. (Each decision tree in the ensemble is only working on a subset of the data, so it can track its accuracy with the data not in its own bag.) 

The accuracy of a classifier is the fraction of predictions made that are correct. This one looks like its doing well - but this is the accuracy on the training set.

### Classifier improvement with GridSearchCV

In [ ]:
# Parameter values to try:
parameters = {'n_estimators':(50,100,200),"max_features": ["auto",3],
              'criterion':["gini","entropy"],"min_samples_leaf": [1,2]}

# Initial training/test split:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# Do a grid search to find the highest 3-fold CV score:
rf_tuned = GridSearchCV(rf, parameters, cv=3, verbose=1)
RFselector = rf_tuned.fit(X_train, y_train)

# Print the best score and estimator:
print(RFselector.best_score_)
print(RFselector.best_estimator_)

#### Question:

Would you be satisfied with a 95% successful classification fraction? Read the Random Forest `SciKit-Learn` docs to find some alternative scores, and think about when you might want to choose one of these instead. (Hint: imagine using a classifier to select a sample of *targets*.)

One way of visualizing classification accuracy is via a *confusion matrix*:

In [ ]:
y_pred = RFselector.predict(X_test)

In [ ]:
# Compute confusion matrix:

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')

Each output label comes with a *classification probability*, computed from the results of the whole forest. To select a sample of classified objects, one can choose a selection threshold in this class probability, and only keep objects with higher probability than this threshold.


The availability of a class probability leads to an important diagnostic: the "Receiver Operating Characteristic" or ["ROC" curve](http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html). This shows the *true positive rate* (TPR) plotted against the *false positive rate* (FPR) of a classifier, as the selection threshold is varied.


Typically, classifiers have control parameters that affect both the TPR and FPR (often improving one at the expense of the other), so the ROC curve is a good tool for investigating these parameters. 


Likewise, ROC curves provide a very good way to compare different classifiers.

### Exercise:

Use `SciKit-Learn` utilities to plot an ROC curve for the RFselector.

**[Back to the lesson plan](../../lessons/9.MachineLearning.ipynb)**